1. Importing libraries 

In [73]:
import re, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, os 

Setting display options

In [74]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

2. Reading the data 

In [75]:
PROJECT_DIR  = "/Users/abhisheksaurav/Desktop/real_estate_project/"
DATA_DIR = "Data Collection/Data/City/chandigarh/cleaned_data"

In [76]:
def get_data(name):
    file_name = "{}.csv".format(name)
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [77]:
df = get_data("gurgaon_properties_cleaned_v1")

In [78]:
df.head(1)

,property type,society,sector,price,price_per_sq_ft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features
0,flat,ats kocoon,sector 109,1.7,9742.0,1745.0,Super Built up area 1745(162.12 sq.m.)Built Up area: 1550 sq.ft. (144 sq.m.),3,3,3,not available,22.0,east,1 to 5 Year Old,"['Jagdish Super Market', 'Dwarka', 'Euro International School', 'Gyaananda School', 'SCR Model School', 'Radha Krishan Mandir', 'ICICI BANK ATM, Annapurna MKT', 'ESIC Dispensary', 'IGIA Airport', 'Canara Bank New Palam Vihar', 'Gurgaon Gramin Bank', 'Daultabad Stadium', 'Bharat Petroleum Shree Shyam Filling', 'The Club, International City', 'Conscient One']","['1 Modular Kitchen', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Light', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Swimming Pool', 'Water Storage', 'Park', 'Visitor Parking', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center']"


4. Detailed Analysis

4.1 areaWithType

In [79]:
df.sample(5)[['price','area', 'areaWithType']]

,price,area,areaWithType
1568,1.10,2077.0,Super Built up area 2077(192.96 sq.m.)Carpet area: 1600 sq.ft. (148.64 sq.m.)
1864,1.33,783.0,Built Up area: 1423 (132.2 sq.m.)Carpet area: 783 sq.ft. (72.74 sq.m.)
998,1.25,1980.0,Super Built up area 1980(183.95 sq.m.)
1455,1.25,82781.0,Plot area 115(7692.86 sq.m.)
1956,1.42,1600.0,Super Built up area 2390(222.04 sq.m.)Built Up area: 2000 sq.ft. (185.81 sq.m.)Carpet area: 1600 sq.ft. (148.64 sq.m.)


In [95]:
# function to extract the Super built up area 

def get_super_built_up_area(text):
    match = re.search(r'Super Built up area (\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [96]:
# function to extract the built up area or carpet area 

def get_area(text, area_type):
    match = re.search(area_type + r'\s*:\s*(\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [99]:
# this function checks if the area is in sq.m and converts to square feet if needed 

def convert_to_sqft(text, area_value):
    if area_value is None:
        return None
    match = re.search(r'{} \((\d+\.?\d*) sq.m.\)'.format(area_value), text)
    if match:
        sq_m_value = float(match.group(1))
        return sq_m_value * 10.7639  # conversion factor from sq.m. to sqft
    return area_value


In [100]:
# extracting super built up area and converting it into sq_ft if needed 

df['super_built_up_area'] = df.areaWithType.apply(get_super_built_up_area)
df['super_built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['super_built_up_area']), axis=1)

In [72]:
# extracting built up area and converting it if needed 

